# Ομαδοποίηση των χρηστών αναλογώς το πως βαθμολογούν το κάθε είδος ταινίας και τα χαρακτηριστικά τους (με αλγόριθμο KMeans)

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

## Import Datasets

In [2]:
# Movies
movies_df = pd.read_csv('data/movies.csv', sep='\t')
movies_df = movies_df.drop(movies_df.columns[0], axis=1)

# Users
users_df = pd.read_csv('data/users.csv', sep='\t')
users_df = users_df.drop(users_df.columns[0], axis=1)

# Ratings
ratings_df = pd.read_csv('data/ratings.csv', sep=';')
ratings_df = ratings_df.drop(ratings_df.columns[0], axis=1)

## Βρίσκουμε το average rating κάθε είδους για κάθε χρήστη

### User ids

In [3]:
user_ids = np.array(users_df.user_id)
user_ids

array([   1,    2,    3, ..., 6038, 6039, 6040])

### Παράδειγμα για 1 user

In [4]:
test_id = 1

ratings_curr_user_df = ratings_df[ratings_df.user_id == test_id]
ratings_curr_user_df.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
# merge με τα movies
# merge right για να πάρουμε μόνο τις ταινίες που έχουν rating
curr_user_merged_df = movies_df.merge(ratings_curr_user_df, on='movie_id', how='right')
curr_user_merged_df.head()

,movie_id,title,genres,user_id,rating,timestamp
0,1193,One Flew Over the Cuckoo's Nest (1975),Drama,1,5,978300760
1,661,James and the Giant Peach (1996),Animation|Children's|Musical,1,3,978302109
2,914,My Fair Lady (1964),Musical|Romance,1,3,978301968
3,3408,Erin Brockovich (2000),Drama,1,4,978300275
4,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy,1,5,978824291


### Θέλουμε να χωρίσουμε τα Genres

#### Για να κάνουμε split μπορούμε να χρησιμοποιήσουμε τη μέθοδο assign()
-> https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.assign.html

In [6]:
split_genres_test = curr_user_merged_df.assign(Genre=curr_user_merged_df.genres.str.split(r'|'))
split_genres_test.head()

,movie_id,title,genres,user_id,rating,timestamp,Genre
0,1193,One Flew Over the Cuckoo's Nest (1975),Drama,1,5,978300760,[Drama]
1,661,James and the Giant Peach (1996),Animation|Children's|Musical,1,3,978302109,"[Animation, Children's, Musical]"
2,914,My Fair Lady (1964),Musical|Romance,1,3,978301968,"[Musical, Romance]"
3,3408,Erin Brockovich (2000),Drama,1,4,978300275,[Drama]
4,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy,1,5,978824291,"[Animation, Children's, Comedy]"


#### Μπορούμε να δημιουργήσουμε rows από array ενός column με την explode()
-> https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html

In [7]:
split_genres_test.explode('Genre').head()

,movie_id,title,genres,user_id,rating,timestamp,Genre
0,1193,One Flew Over the Cuckoo's Nest (1975),Drama,1,5,978300760,Drama
1,661,James and the Giant Peach (1996),Animation|Children's|Musical,1,3,978302109,Animation
1,661,James and the Giant Peach (1996),Animation|Children's|Musical,1,3,978302109,Children's
1,661,James and the Giant Peach (1996),Animation|Children's|Musical,1,3,978302109,Musical
2,914,My Fair Lady (1964),Musical|Romance,1,3,978301968,Musical


#### Επίσης μπορούμε να πάρουμε όλα τα διαφορετικά genres

In [8]:
movies_df["genres"].str.split('|').explode()

0        Animation
0       Children's
0           Comedy
1        Adventure
1       Children's
           ...    
3879         Drama
3880         Drama
3881         Drama
3882         Drama
3882      Thriller
Name: genres, Length: 6408, dtype: object

In [9]:
unique_genres = movies_df["genres"].str.split('|').explode().unique()
unique_genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

### Οπότε έχουμε:

In [10]:
# Σπάμε το column genres
avg_ratings_per_genre_curr_user_df = curr_user_merged_df.assign(Genre=curr_user_merged_df.genres.str.split(r'|')).explode('Genre')
avg_ratings_per_genre_curr_user_df.head(7)

,movie_id,title,genres,user_id,rating,timestamp,Genre
0,1193,One Flew Over the Cuckoo's Nest (1975),Drama,1,5,978300760,Drama
1,661,James and the Giant Peach (1996),Animation|Children's|Musical,1,3,978302109,Animation
1,661,James and the Giant Peach (1996),Animation|Children's|Musical,1,3,978302109,Children's
1,661,James and the Giant Peach (1996),Animation|Children's|Musical,1,3,978302109,Musical
2,914,My Fair Lady (1964),Musical|Romance,1,3,978301968,Musical
2,914,My Fair Lady (1964),Musical|Romance,1,3,978301968,Romance
3,3408,Erin Brockovich (2000),Drama,1,4,978300275,Drama


In [11]:
# Δεν θα χρειαστούμε τον τίτλο, τα timestamps και τα αρχικά genres
avg_ratings_per_genre_curr_user_df.drop(['genres', 'timestamp', 'title'], axis=1, inplace=True)
avg_ratings_per_genre_curr_user_df

,movie_id,user_id,rating,Genre
0,1193,1,5,Drama
1,661,1,3,Animation
1,661,1,3,Children's
1,661,1,3,Musical
2,914,1,3,Musical
...,...,...,...,...
50,3114,1,4,Comedy
51,608,1,4,Crime
51,608,1,4,Drama
51,608,1,4,Thriller


In [12]:
# Βρίσκουμε τα average ratings χρησιμοποιώντας το function mean()
avg_ratings_per_genre_curr_user_df = avg_ratings_per_genre_curr_user_df.groupby('Genre').rating.mean()
avg_ratings_per_genre_curr_user_df

Genre
Action        4.200000
Adventure     4.000000
Animation     4.111111
Children's    4.250000
Comedy        4.142857
Crime         4.000000
Drama         4.428571
Fantasy       4.000000
Musical       4.285714
Romance       3.666667
Sci-Fi        4.333333
Thriller      3.666667
War           5.000000
Name: rating, dtype: float64

#### Βάζουμε όλα τα genres (μπορεί ο χρήστης να μην έχει βαθμολογίες σε μερικά genres... τα κάνουμε fill με 0)
-> https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reindex.html

In [13]:
avg_ratings_per_genre_curr_user_df = avg_ratings_per_genre_curr_user_df.reindex(unique_genres).fillna(0)
avg_ratings_per_genre_curr_user_df.sort_index(inplace=True) # sort για να έχουν όλοι οι χρήστες την ίδια σειρά
avg_ratings_per_genre_curr_user_df

Genre
Action         4.200000
Adventure      4.000000
Animation      4.111111
Children's     4.250000
Comedy         4.142857
Crime          4.000000
Documentary    0.000000
Drama          4.428571
Fantasy        4.000000
Film-Noir      0.000000
Horror         0.000000
Musical        4.285714
Mystery        0.000000
Romance        3.666667
Sci-Fi         4.333333
Thriller       3.666667
War            5.000000
Western        0.000000
Name: rating, dtype: float64

### Φτιάχνουμε το array που θα κάνουμε fit στον KMeans
(πρέπει να το κάνουμε για κάθε χρήστη και στη συνέχεια να τα ενώσουμε)

In [14]:
kmeans_ratings_data = np.array(avg_ratings_per_genre_curr_user_df.values)
kmeans_ratings_data

array([4.2       , 4.        , 4.11111111, 4.25      , 4.14285714,
       4.        , 0.        , 4.42857143, 4.        , 0.        ,
       0.        , 4.28571429, 0.        , 3.66666667, 4.33333333,
       3.66666667, 5.        , 0.        ])

In [15]:
# Θέλουμε να βάλουμε και τα χαρακτηριστικά του χρήστη
user_data = users_df[users_df.user_id == test_id]
user_data

,user_id,gender,age,occupation,zipcode,age_desc,occ_desc
0,1,F,1,10,48067,Under 18,K-12 student


#### Μετατρέπουμε τα columns gender, age_desc σε categorical data

In [16]:
new_users_df = users_df
new_users_df.gender = new_users_df.gender.astype('category')
new_users_df.age_desc = new_users_df.age_desc.astype('category')
new_users_df.head()

,user_id,gender,age,occupation,zipcode,age_desc,occ_desc
0,1,F,1,10,48067,Under 18,K-12 student
1,2,M,56,16,70072,56+,self-employed
2,3,M,25,15,55117,25-34,scientist
3,4,M,45,7,02460,45-49,executive/managerial
4,5,M,25,20,55455,25-34,writer


In [17]:
curr_user_data = new_users_df[new_users_df.user_id == test_id]
curr_user_data

,user_id,gender,age,occupation,zipcode,age_desc,occ_desc
0,1,F,1,10,48067,Under 18,K-12 student


### Παίρνουμε το gender, occupation, age_desc

In [18]:
kmeans_curr_user_data = np.append(kmeans_ratings_data, curr_user_data.gender.cat.codes) # gender categorical code
kmeans_curr_user_data = np.append(kmeans_curr_user_data, curr_user_data.occupation) # occupation (already coded)
kmeans_curr_user_data = np.append(kmeans_curr_user_data, curr_user_data.age_desc.cat.codes) # age description categorical code
kmeans_curr_user_data

array([ 4.2       ,  4.        ,  4.11111111,  4.25      ,  4.14285714,
        4.        ,  0.        ,  4.42857143,  4.        ,  0.        ,
        0.        ,  4.28571429,  0.        ,  3.66666667,  4.33333333,
        3.66666667,  5.        ,  0.        ,  0.        , 10.        ,
        6.        ])

### **Θα πρέπει να το κάνουμε για κάθε χρήστη και τέλος να τα βάλουμε σε ένα array όλα μαζί

In [19]:
kmeans_complete = []
kmeans_complete.append(kmeans_curr_user_data)
kmeans_complete

[array([ 4.2       ,  4.        ,  4.11111111,  4.25      ,  4.14285714,
         4.        ,  0.        ,  4.42857143,  4.        ,  0.        ,
         0.        ,  4.28571429,  0.        ,  3.66666667,  4.33333333,
         3.66666667,  5.        ,  0.        ,  0.        , 10.        ,
         6.        ])]

In [20]:
# Random data απλά για να δοκιμάσουμε τη λειτουργία του KMeans
dummy_data = np.random.rand(21)
kmeans_complete.append(dummy_data)
kmeans_complete

[array([ 4.2       ,  4.        ,  4.11111111,  4.25      ,  4.14285714,
         4.        ,  0.        ,  4.42857143,  4.        ,  0.        ,
         0.        ,  4.28571429,  0.        ,  3.66666667,  4.33333333,
         3.66666667,  5.        ,  0.        ,  0.        , 10.        ,
         6.        ]),
 array([0.12926058, 0.16067352, 0.11277938, 0.45035444, 0.77362811,
        0.86880245, 0.24036277, 0.71680226, 0.07220089, 0.3634009 ,
        0.40638678, 0.00899036, 0.10798724, 0.73774377, 0.47526013,
        0.84632498, 0.95937888, 0.1903338 , 0.8079683 , 0.93221238,
        0.95147036])]

## Implement τον KMeans

In [21]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(kmeans_complete)

KMeans(n_clusters=2)

In [22]:
# Prediction σε ποια 'ομάδα' ανήκει κάθε χρήστης
y_kmeans = kmeans.predict(kmeans_complete)
y_kmeans

array([1, 0], dtype=int32)

# Πρέπει να κάνουμε την διαδικασία αυτή για όλους τους χρήστες

In [23]:
def get_average_user_ratings_by_genre(id):
    ratings_curr_user_df = ratings_df[ratings_df.user_id == id]
    curr_user_merged_df = movies_df.merge(ratings_curr_user_df, on='movie_id', how='right')
    ratings_per_genre_curr_user_df = curr_user_merged_df.assign(Genre=curr_user_merged_df.genres.str.split(r'|')).explode('Genre')
    avg_ratings_per_genre_curr_user = ratings_per_genre_curr_user_df.groupby('Genre').rating.mean()
    avg_ratings_per_genre_curr_user = avg_ratings_per_genre_curr_user.reindex(unique_genres).fillna(0)
    avg_ratings_per_genre_curr_user.sort_index(inplace=True)
    return np.array(avg_ratings_per_genre_curr_user.values)

In [24]:
def get_user_info(id):
    curr_user_data = users_df[users_df.user_id == id]
    user_info = np.append(curr_user_data.gender.cat.codes, curr_user_data.occupation)
    user_info = np.append(user_info, curr_user_data.age_desc.cat.codes)
    return user_info

In [25]:
user_ids = np.array(users_df.user_id) # Πάιρνουμε array με όλα τα ids από το users dataframe
kmeans_complete_dataset = [] # Initialize το array

# Loop για κάθε user και δημιουργία array με τα ratings του χρήστη σε κάθε είδος ΚΑΙ τα βασικά χαρακτηριστικά του (age, occupation etc...)
for user_id in user_ids:
    kmeans_avg_ratings_dataset = get_average_user_ratings_by_genre(user_id)
    kmeans_user_info_dataset = get_user_info(user_id)

    kmeans_curr_user_dataset = np.append(kmeans_avg_ratings_dataset, kmeans_user_info_dataset)
    kmeans_complete_dataset.append(kmeans_curr_user_dataset)

In [26]:
kmeans_complete_dataset

[array([ 4.2       ,  4.        ,  4.11111111,  4.25      ,  4.14285714,
         4.        ,  0.        ,  4.42857143,  4.        ,  0.        ,
         0.        ,  4.28571429,  0.        ,  3.66666667,  4.33333333,
         3.66666667,  5.        ,  0.        ,  0.        , 10.        ,
         6.        ]),
 array([ 3.5       ,  3.73684211,  0.        ,  0.        ,  3.56      ,
         3.58333333,  0.        ,  3.89873418,  3.        ,  4.        ,
         3.        ,  0.        ,  3.33333333,  3.70833333,  3.58823529,
         3.48387097,  3.73333333,  4.33333333,  1.        , 16.        ,
         5.        ]),
 array([ 3.95652174,  4.        ,  4.        ,  4.        ,  3.76666667,
         0.        ,  0.        ,  4.        ,  4.5       ,  0.        ,
         2.66666667,  4.        ,  3.        ,  3.8       ,  3.83333333,
         3.8       ,  4.        ,  4.66666667,  1.        , 15.        ,
         1.        ]),
 array([4.15789474, 3.83333333, 0.        , 4.        ,

## Εφαρμόζουμε τον kmeans με το ολοκληρωμένο dataset αυτή τη φορά

In [27]:
kmeans = KMeans(n_clusters=5) # δοκιμάζουμε με 5 clusters
kmeans.fit(kmeans_complete_dataset)

KMeans(n_clusters=5)

In [28]:
predictions = kmeans.predict(kmeans_complete_dataset)
predictions

array([3, 4, 1, ..., 2, 0, 3], dtype=int32)

In [29]:
predictions.size

6040

### Θέλουμε να αντιστοιχίσουμε τους users με την ομάδα στην οποία ανήκουν (από τις προβλέψεις του kmeans)

In [32]:
users_clustered = pd.DataFrame(columns=['user_id', 'cluster'])

In [33]:
i = 0
for uid in user_ids:
    users_clustered = users_clustered.append({'user_id': uid, 'cluster': predictions[i]}, ignore_index=True)
    i += 1

In [34]:
users_clustered

,user_id,cluster
0,1,3
1,2,4
2,3,1
3,4,3
4,5,1
...,...,...
6035,6036,1
6036,6037,0
6037,6038,2
6038,6039,0


### Μπορούμε τώρα να βρούμε όλους τους χρήστες που ανήκουν στο ίδιο cluster

In [35]:
# πχ cluster για τον user με id = 23
users_clustered[users_clustered['user_id']==23]

,user_id,cluster
22,23,0


In [36]:
num = users_clustered[users_clustered['user_id']==23].cluster.values[0]
num

0

In [37]:
# users στο ίδιο cluster
users_same_cluster = users_clustered[users_clustered.cluster == num]
users_same_cluster

,user_id,cluster
9,10,0
12,13,0
15,16,0
16,17,0
17,18,0
...,...,...
6022,6023,0
6024,6025,0
6030,6031,0
6036,6037,0


In [38]:
# id των user το ίδο cluster
user_ids_same_cluster = users_same_cluster.user_id.values
user_ids_same_cluster

array([10, 13, 16, ..., 6031, 6037, 6039], dtype=object)

### Χρησιμοποιώντας τα ids των user που βρίσκονται στην ίδια ομάδα, μπορούμε να πάρουμε όλες τις βαθμολογίες τους από το  ratings_df

In [39]:
related_users_ratings = pd.DataFrame()
for user_id in user_ids_same_cluster:
    current_ratings = ratings_df[ratings_df['user_id']==user_id].drop(['timestamp'], axis=1)
    related_users_ratings = related_users_ratings.append(current_ratings)

In [40]:
related_users_ratings

,user_id,movie_id,rating
799,10,2622,5
800,10,648,4
801,10,2628,3
802,10,3358,5
803,10,3359,3
...,...,...,...
999863,6039,1081,4
999864,6039,1083,3
999865,6039,1086,4
999866,6039,1088,4


In [41]:
# Αριθμός βαθμολογιών για κάθε ταινία
movie_id_counts = related_users_ratings.movie_id.value_counts()
movie_id_counts

2858    1138
1196    1065
260     1056
1210     964
2571     929
        ... 
1868       1
3151       1
3523       1
2673       1
2895       1
Name: movie_id, Length: 3604, dtype: int64

In [42]:
# Ταινίες με τουλάχιστον 10 βαθμολογίες από διαφορετικούς χρήστες
frequently_rated_movies = movie_id_counts[movie_id_counts >= 10]
frequently_rated_movies

2858    1138
1196    1065
260     1056
1210     964
2571     929
        ... 
3580      10
3520      10
3222      10
3473      10
2678      10
Name: movie_id, Length: 2972, dtype: int64

In [43]:
# array με ids ταινιών με τουλάχιστων 10 βαθμολογίες από διαφορετικούς χρήστες
frequently_rated_movies_ids = frequently_rated_movies.index
frequently_rated_movies_ids

Int64Index([2858, 1196,  260, 1210, 2571, 1270,  593,  589,  480, 2028,
            ...
            3781,  525, 2809,  274, 1812, 3580, 3520, 3222, 3473, 2678],
           dtype='int64', length=2972)

#### Διαγράφουμε τις βαθμολογίες των ταινιών που έχουν βαθμολογηθεί από <10 χρήστες (για να μην χαλάσει το average) 
-> https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html

In [44]:
related_users_ratings = related_users_ratings[related_users_ratings.movie_id.isin(frequently_rated_movies_ids)]
related_users_ratings

,user_id,movie_id,rating
799,10,2622,5
800,10,648,4
801,10,2628,3
802,10,3358,5
803,10,3359,3
...,...,...,...
999863,6039,1081,4
999864,6039,1083,3
999865,6039,1086,4
999866,6039,1088,4


### Έτσι μπορούμε να βρούμε την average βαθμολογία μιας ταινίας σύμφωνα με τις βαθμολογίες των χρηστών σε ένα συγκεκριμένο cluster

In [45]:
related_users_avg_ratings_per_movie = related_users_ratings.groupby('movie_id').rating.mean()
related_users_avg_ratings_per_movie

movie_id
1       4.196154
2       3.183051
3       2.903409
4       2.758621
5       3.094017
          ...   
3948    3.765306
3949    4.107143
3950    3.400000
3951    4.133333
3952    3.623077
Name: rating, Length: 2972, dtype: float64

### Τέλος βρίσκουμε για ένα συγκεκριμένο χρήστη τις ταινίες τις οποίες ΔΕΝ έχει βαθμολογήσει και συμπληρώνουμε τις βαθμολογίες αυτές με το average rating των χρηστών που ανήκουν στο ίδιο cluster

In [46]:
# ταινίες που ΔΕΝ έχει βαθμολογήσει ο χρήστης με id = 23 (για παράδειγμα)
user_complete_ratings = ratings_df[ratings_df['user_id'] == 23]
user_ratings_merged = movies_df.merge(user_complete_ratings, on='movie_id', how='left').drop(['user_id', 'timestamp', 'genres'], axis=1)
user_incomplete_ratings = user_ratings_merged[user_ratings_merged.rating.isna()]

user_incomplete_ratings

,movie_id,title,rating
2,3,Grumpier Old Men (1995),NaN
3,4,Waiting to Exhale (1995),NaN
4,5,Father of the Bride Part II (1995),NaN
6,7,Sabrina (1995),NaN
7,8,Tom and Huck (1995),NaN
...,...,...,...
3877,3947,Get Carter (1971),NaN
3878,3948,Meet the Parents (2000),NaN
3879,3949,Requiem for a Dream (2000),NaN
3880,3950,Tigerland (2000),NaN


#### Συμπληρώνουμε όλες τις βαθμολογίες από το related_users_avg_ratings_per_movie
-> https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html

-> https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html

In [47]:
type(related_users_avg_ratings_per_movie)

pandas.core.series.Series

In [48]:
# Βάζουμε σαν index το movie_id εφόσον το αντικείμενο related_users_avg_ratings_per_movie είναι pandas.Series με indexes τα movie ids
user_updated_ratings = user_incomplete_ratings.set_index('movie_id').rating.fillna(related_users_avg_ratings_per_movie).reset_index()
user_updated_ratings

,movie_id,rating
0,3,2.903409
1,4,2.758621
2,5,3.094017
3,7,3.360656
4,8,2.935484
...,...,...
3574,3947,3.437500
3575,3948,3.765306
3576,3949,4.107143
3577,3950,3.400000


## Χρησιμοποιώντας τα updated ratings μπορούμε να τα κάνουμε sort και να δώσουμε στο χρήστη recommendations

In [49]:
user_updated_ratings = user_updated_ratings.sort_values(by='rating', ascending=False)
user_updated_ratings.head(10)

,movie_id,rating
2598,2905,4.666667
606,669,4.666667
1786,2019,4.568093
838,922,4.560976
285,318,4.554133
607,670,4.500000
479,527,4.493333
1088,1198,4.480984
1074,1178,4.463918
291,326,4.448276


In [50]:
# Merge για να πάρουμε και τους τίτλους των ταινιών
final_result = user_updated_ratings.merge(movies_df, on='movie_id', how='left')
final_result.head(10)

,movie_id,rating,title,genres
0,2905,4.666667,Sanjuro (1962),Action|Adventure
1,669,4.666667,Aparajito (1956),Drama
2,2019,4.568093,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama
3,922,4.560976,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Film-Noir
4,318,4.554133,"Shawshank Redemption, The (1994)",Drama
5,670,4.500000,"World of Apu, The (Apur Sansar) (1959)",Drama
6,527,4.493333,Schindler's List (1993),Drama|War
7,1198,4.480984,Raiders of the Lost Ark (1981),Action|Adventure
8,1178,4.463918,Paths of Glory (1957),Drama|War
9,326,4.448276,To Live (Huozhe) (1994),Drama
